In [1]:
%%capture
%pip install sqlalchemy aiosqlite aiocache

In [2]:
import asyncio
from aiocache import RedisCache
from aiocache.serializers import PickleSerializer

# Set up the Redis cache with a PickleSerializer
cache = RedisCache(
    endpoint="127.0.0.1",
    port=6379,
    serializer=PickleSerializer(),
    ttl=60  # Default TTL of 60 seconds
)

In [ ]:
import asyncio
import hashlib
import pickle
from sqlalchemy import Column, Integer, String, select
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import declarative_base
from sqlalchemy.engine import Result
from sqlalchemy.ext.asyncio import async_sessionmaker

Base = declarative_base()

# Example model
class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable=False)

# Custom AsyncSession class to override 'execute'
class CustomAsyncSession(AsyncSession):
    async def execute(self, statement, *args, **kwargs) -> Result:
        print("Custom execute called with:", statement)
        # serialized_obj = pickle.dumps(my_object, protocol=pickle.HIGHEST_PROTOCOL)

        # Ensure we have a dialect to compile against
        if self.bind is None:
            raise RuntimeError("No bind found on session. Make sure your session is properly initialized.")

        # Compile the statement into a SQL string
        compiled = statement.compile(
            dialect=self.bind.dialect,
            compile_kwargs={"literal_binds": True}
        )
        query_str = str(compiled)

        # Generate a unique hash for the query
        query_hash = hashlib.sha256(query_str.encode("utf-8")).hexdigest()
        if result:= await cache.get(query_hash):
            return result
        db_result = await super().execute(statement, *args, **kwargs)
        await cache.set(query_hash,pickle.dumps(db_result))
        # Do something with the query_hash, e.g., logging
        print(f"Executing async query with hash: {query_hash}")
        print(f"Compiled query: {query_str}")

        # Call the original execute method
        return db_result

# Create an async engine
engine = create_async_engine("sqlite+aiosqlite:///:memory:", echo=False, future=True)

# Create a session factory using our custom session class
async_session_factory = async_sessionmaker(
    engine, expire_on_commit=False, class_=CustomAsyncSession, autoflush=False, future=True
)


In [ ]:
async def async_setup():
    async with engine.begin() as conn:
        await conn.run_sync(Base.metadata.create_all)
    
    # Insert some sample data
    async with async_session_factory() as session:
        session.add_all([
            User(name="Alice"),
            User(name="Bob"),
            User(name="Charlie")
        ])
        await session.commit()

# Run the setup
await async_setup()


In [ ]:
async def async_query():
    async with async_session_factory() as session:
        # This will use the overridden execute method
        result = await session.execute(select(User))
        users = result.scalars().all()
        return users

users = await async_query()
users


Custom execute called with: SELECT users.id, users.name 
FROM users


In [ ]:

# Helper function to get a cache key for a query
def get_cache_key(clauseelement, params):
    # Create a unique cache key by hashing the SQL query and params
    query_str = str(clauseelement) + json.dumps(params, sort_keys=True)
    return hashlib.md5(query_str.encode('utf-8')).hexdigest()
